# setup and configuration

In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os
import pickle

# --- Setup ---
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
plt.style.use('seaborn-v0_8-darkgrid')

# --- Random Seed ---
RANDOM_STATE = 123
np.random.seed(RANDOM_STATE)
os.environ["PYTHONHASHSEED"] = str(RANDOM_STATE)

# --- Model/Metrics ---
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.metrics import f1_score, classification_report, confusion_matrix, roc_auc_score
from sklearn.metrics import precision_recall_curve, average_precision_score, precision_score, recall_score
from sklearn.linear_model import LogisticRegression
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

# --- ML Models ---
import lightgbm as lgb
import xgboost as xgb

# --- Tuning ---
try:
    import optuna
except ImportError:
    print("Installing Optuna...")
    !pip install optuna -q
    import optuna

optuna.logging.set_verbosity(optuna.logging.WARNING)

# --- Feature Engineering Helpers ---
# REMOVED pyzipcode - we won't use 'state' feature due to data quality issues
# try:
#     from pyzipcode import ZipCodeDatabase
# except ImportError:
#     print("Installing pyzipcode...")
#     !pip install pyzipcode -q
#     from pyzipcode import ZipCodeDatabase

print("\n✓ Setup complete. All libraries loaded.")


✓ Setup complete. All libraries loaded.


# data loading


In [28]:
# --- Load Data ---
# Check if running in Google Colab
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB:
    from google.colab import files
    try:
        train_df = pd.read_csv('Training_TriGuard.csv')
        test_df = pd.read_csv('Testing_TriGuard.csv')
        print("✓ Files loaded from local environment.")
    except FileNotFoundError:
        print("Please upload Training_TriGuard.csv:")
        uploaded_train = files.upload()
        train_file = list(uploaded_train.keys())[0]
        train_df = pd.read_csv(train_file)

        print("\nPlease upload Testing_TriGuard.csv:")
        uploaded_test = files.upload()
        test_file = list(uploaded_test.keys())[0]
        test_df = pd.read_csv(test_file)
        print("✓ Files uploaded successfully.")
else:
    # Local environment
    possible_paths = [
        'Training_TriGuard.csv',
        'data/Training_TriGuard.csv',
        '../Training_TriGuard.csv',
        './Training_TriGuard.csv'
    ]

    train_path = None
    test_path = None

    for path in possible_paths:
        if os.path.exists(path):
            train_path = path
            test_path = path.replace('Training', 'Testing')
            if os.path.exists(test_path):
                break

    if train_path and test_path:
        train_df = pd.read_csv(train_path)
        test_df = pd.read_csv(test_path)
        print(f"✓ Files loaded from: {train_path} and {test_path}")
    else:
        train_path = input("Enter path to Training_TriGuard.csv: ").strip()
        test_path = input("Enter path to Testing_TriGuard.csv: ").strip()
        train_df = pd.read_csv(train_path)
        test_df = pd.read_csv(test_path)
        print("✓ Files loaded successfully.")

# --- Critical Cleaning ---
initial_train_count = len(train_df)
train_df = train_df.dropna(subset=['subrogation'])
print(f"\nCleaned training data: Removed {initial_train_count - len(train_df)} rows with NaN target.")

train_df['subrogation'] = train_df['subrogation'].astype(int)

print(f"\n✓ Train shape: {train_df.shape}")
print(f"✓ Test shape: {test_df.shape}")
print(f"\nTarget distribution (after cleaning):")
print(train_df['subrogation'].value_counts(normalize=True).to_string())

test_ids = test_df['claim_number'].copy()

print("✓ Data loading complete.")

✓ Files loaded from local environment.

Cleaned training data: Removed 2 rows with NaN target.

✓ Train shape: (17999, 29)
✓ Test shape: (12000, 28)

Target distribution (after cleaning):
subrogation
0   0.771
1   0.229
✓ Data loading complete.


# feature engineering


In [29]:
def feature_engineer(df):
    """Feature engineering WITHOUT vehicle_made_year/vehicle_age/state (data quality issues)"""
    df_fe = df.copy()

    # ========================================================================
    # TEMPORAL FEATURES
    # ========================================================================
    df_fe['claim_date'] = pd.to_datetime(df_fe['claim_date'], errors='coerce')
    df_fe['claim_year'] = df_fe['claim_date'].dt.year
    df_fe['claim_month'] = df_fe['claim_date'].dt.month
    df_fe['claim_day'] = df_fe['claim_date'].dt.day
    df_fe['claim_quarter'] = df_fe['claim_date'].dt.quarter
    df_fe['claim_dayofweek'] = df_fe['claim_date'].dt.dayofweek
    df_fe['is_weekend'] = (df_fe['claim_dayofweek'] >= 5).astype(int)
    df_fe['is_monday'] = (df_fe['claim_dayofweek'] == 0).astype(int)
    df_fe['is_friday'] = (df_fe['claim_dayofweek'] == 4).astype(int)
    df_fe['is_q4'] = (df_fe['claim_quarter'] == 4).astype(int)

    # NEW: Time-of-day features from Doc 8
    df_fe['claim_hour'] = df_fe['claim_date'].dt.hour
    df_fe['rush_hour'] = df_fe['claim_hour'].isin([7, 8, 9, 16, 17, 18]).astype(int)
    df_fe['late_night'] = df_fe['claim_hour'].isin([0, 1, 2, 3, 4, 5]).astype(int)

    season_map = {
        3: 'Spring', 4: 'Spring', 5: 'Spring',
        6: 'Summer', 7: 'Summer', 8: 'Summer',
        9: 'Fall', 10: 'Fall', 11: 'Fall',
        12: 'Winter', 1: 'Winter', 2: 'Winter'
    }
    df_fe['season'] = df_fe['claim_month'].map(season_map).fillna('Unknown')

    # ========================================================================
    # DATA CLEANING
    # ========================================================================
    df_fe.loc[(df_fe['year_of_born'] < 1900) | (df_fe['year_of_born'] > 2025), 'year_of_born'] = np.nan

    # ========================================================================
    # BINARY CONVERSIONS (for interactions)
    # ========================================================================
    df_fe['witness_binary'] = (df_fe['witness_present_ind'] == 'Y').astype(int)
    df_fe['police_binary'] = df_fe['policy_report_filed_ind']
    df_fe['multicar_binary'] = df_fe['accident_type'].isin(['multi_vehicle_clear', 'multi_vehicle_unclear']).astype(int)
    df_fe['highrisk_site_binary'] = df_fe['accident_site'].isin(['Highway/Intersection', 'Local']).astype(int)

    # ========================================================================
    # CRITICAL INTERACTION FEATURES (2-way)
    # ========================================================================
    df_fe['liab_x_witness'] = df_fe['liab_prct'] * df_fe['witness_binary']
    df_fe['liab_x_police'] = df_fe['liab_prct'] * df_fe['police_binary']
    df_fe['liab_x_multicar'] = df_fe['liab_prct'] * df_fe['multicar_binary']
    df_fe['liab_x_highrisk_site'] = df_fe['liab_prct'] * df_fe['highrisk_site_binary']
    df_fe['liab_x_evidence'] = df_fe['liab_prct'] * (df_fe['witness_binary'] + df_fe['police_binary'])
    df_fe['liab_x_payout'] = df_fe['liab_prct'] * df_fe['claim_est_payout']
    df_fe['liab_x_mileage'] = df_fe['liab_prct'] * df_fe['vehicle_mileage']

    df_fe['witness_x_police'] = df_fe['witness_binary'] * df_fe['police_binary']
    df_fe['witness_x_multicar'] = df_fe['witness_binary'] * df_fe['multicar_binary']
    df_fe['police_x_multicar'] = df_fe['police_binary'] * df_fe['multicar_binary']
    df_fe['multicar_x_highrisk'] = df_fe['multicar_binary'] * df_fe['highrisk_site_binary']
    df_fe['weekend_highway'] = (df_fe['claim_dayofweek'] >= 5).astype(int) * (df_fe['accident_site'] == 'Highway/Intersection').astype(int)

    # 3-way interaction
    df_fe['witness_police_multicar'] = df_fe['witness_binary'] * df_fe['police_binary'] * df_fe['multicar_binary']

    # ========================================================================
    # POLYNOMIAL FEATURES (liability & key variables)
    # ========================================================================
    df_fe['liab_prct_squared'] = df_fe['liab_prct'] ** 2
    df_fe['liab_prct_cubed'] = df_fe['liab_prct'] ** 3
    df_fe['liab_prct_sqrt'] = np.sqrt(df_fe['liab_prct'])
    df_fe['liab_prct_log'] = np.log1p(df_fe['liab_prct'])
    df_fe['liab_inverse'] = 100 - df_fe['liab_prct']
    df_fe['liab_inverse_squared'] = (100 - df_fe['liab_prct']) ** 2

    df_fe['log_claim_est_payout'] = np.log1p(df_fe['claim_est_payout'])
    df_fe['log_vehicle_mileage'] = np.log1p(df_fe['vehicle_mileage'])
    df_fe['log_vehicle_price'] = np.log1p(df_fe['vehicle_price'])
    df_fe['log_annual_income'] = np.log1p(df_fe['annual_income'])
    df_fe['sqrt_vehicle_mileage'] = np.sqrt(df_fe['vehicle_mileage'])

    # ========================================================================
    # ACCIDENT TYPE FEATURES
    # ========================================================================
    df_fe['is_multi_vehicle_clear'] = (df_fe['accident_type'] == 'multi_vehicle_clear').astype(int)
    df_fe['is_multi_vehicle_unclear'] = (df_fe['accident_type'] == 'multi_vehicle_unclear').astype(int)
    df_fe['is_single_car'] = (df_fe['accident_type'] == 'single_car').astype(int)
    df_fe['has_recovery_target'] = df_fe['multicar_binary']

    df_fe['recovery_case_clarity'] = 0
    df_fe.loc[df_fe['is_multi_vehicle_clear'] == 1, 'recovery_case_clarity'] = 3
    df_fe.loc[df_fe['is_multi_vehicle_unclear'] == 1, 'recovery_case_clarity'] = 1

    # ========================================================================
    # LIABILITY BUCKETS (fine-grained)
    # ========================================================================
    df_fe['liab_under_10'] = (df_fe['liab_prct'] < 10).astype(int)
    df_fe['liab_10_to_15'] = ((df_fe['liab_prct'] >= 10) & (df_fe['liab_prct'] < 15)).astype(int)
    df_fe['liab_15_to_20'] = ((df_fe['liab_prct'] >= 15) & (df_fe['liab_prct'] < 20)).astype(int)
    df_fe['liab_20_to_25'] = ((df_fe['liab_prct'] >= 20) & (df_fe['liab_prct'] < 25)).astype(int)
    df_fe['liab_25_to_30'] = ((df_fe['liab_prct'] >= 25) & (df_fe['liab_prct'] < 30)).astype(int)
    df_fe['liab_30_to_35'] = ((df_fe['liab_prct'] >= 30) & (df_fe['liab_prct'] < 35)).astype(int)
    df_fe['liab_35_to_40'] = ((df_fe['liab_prct'] >= 35) & (df_fe['liab_prct'] < 40)).astype(int)
    df_fe['liab_40_to_50'] = ((df_fe['liab_prct'] >= 40) & (df_fe['liab_prct'] < 50)).astype(int)
    df_fe['liab_over_50'] = (df_fe['liab_prct'] >= 50).astype(int)

    df_fe['not_at_fault'] = df_fe['liab_under_10']
    df_fe['minimal_fault'] = (df_fe['liab_prct'] < 25).astype(int)
    df_fe['low_fault'] = (df_fe['liab_prct'] < 35).astype(int)
    df_fe['shared_fault'] = ((df_fe['liab_prct'] >= 35) & (df_fe['liab_prct'] < 50)).astype(int)
    df_fe['high_fault'] = (df_fe['liab_prct'] >= 50).astype(int)

    # ========================================================================
    # EVIDENCE QUALITY FEATURES
    # ========================================================================
    df_fe['witness_present'] = df_fe['witness_binary']
    df_fe['police_report'] = df_fe['police_binary']

    df_fe['evidence_none'] = ((df_fe['witness_present'] == 0) & (df_fe['police_report'] == 0)).astype(int)
    df_fe['evidence_weak'] = (((df_fe['witness_present'] == 1) & (df_fe['police_report'] == 0)) |
                              ((df_fe['witness_present'] == 0) & (df_fe['police_report'] == 1))).astype(int)
    df_fe['evidence_strong'] = ((df_fe['witness_present'] == 1) & (df_fe['police_report'] == 1)).astype(int)
    df_fe['evidence_very_strong'] = ((df_fe['witness_present'] == 1) & (df_fe['police_report'] == 1) &
                                      (df_fe['liab_prct'] < 20)).astype(int)
    df_fe['evidence_score'] = df_fe['witness_present'] + df_fe['police_report']

    # ========================================================================
    # ACCIDENT SITE FEATURES
    # ========================================================================
    df_fe['high_risk_site'] = df_fe['highrisk_site_binary']
    df_fe['parking_accident'] = (df_fe['accident_site'] == 'Parking Area').astype(int)
    df_fe['unknown_site'] = (df_fe['accident_site'] == 'Unknown').astype(int)
    df_fe['highway_accident'] = (df_fe['accident_site'] == 'Highway/Intersection').astype(int)
    df_fe['local_accident'] = (df_fe['accident_site'] == 'Local').astype(int)

    # ========================================================================
    # DRIVER AGE & EXPERIENCE
    # ========================================================================
    df_fe['driver_age'] = df_fe['claim_year'] - df_fe['year_of_born']
    df_fe.loc[(df_fe['driver_age'] < 16) | (df_fe['driver_age'] > 100), 'driver_age'] = np.nan

    df_fe['young_driver'] = ((df_fe['driver_age'] >= 16) & (df_fe['driver_age'] <= 25)).astype(int)
    df_fe['prime_driver'] = ((df_fe['driver_age'] > 25) & (df_fe['driver_age'] <= 45)).astype(int)
    df_fe['middle_age_driver'] = ((df_fe['driver_age'] > 45) & (df_fe['driver_age'] <= 65)).astype(int)
    df_fe['senior_driver'] = (df_fe['driver_age'] > 65).astype(int)

    df_fe['driving_experience'] = (df_fe['driver_age'] - df_fe['age_of_DL']).clip(lower=0)
    df_fe.loc[df_fe['driving_experience'] < 0, 'driving_experience'] = np.nan

    df_fe['novice_driver'] = (df_fe['driving_experience'] < 3).astype(int)
    df_fe['experienced_driver'] = ((df_fe['driving_experience'] >= 3) & (df_fe['driving_experience'] <= 10)).astype(int)
    df_fe['veteran_driver'] = (df_fe['driving_experience'] > 10).astype(int)

    df_fe['experience_x_safety'] = df_fe['driving_experience'] * df_fe['safety_rating']
    df_fe['driver_age_x_safety'] = df_fe['driver_age'] * df_fe['safety_rating']

    # NEW: Driver risk interactions from Doc 8
    df_fe['young_novice'] = df_fe['young_driver'] * df_fe['novice_driver']

    # ========================================================================
    # VEHICLE FEATURES (without vehicle_age)
    # ========================================================================
    df_fe['luxury_vehicle'] = (df_fe['vehicle_price'] > 50000).astype(int)
    df_fe['mid_price_vehicle'] = ((df_fe['vehicle_price'] >= 20000) & (df_fe['vehicle_price'] <= 50000)).astype(int)
    df_fe['economy_vehicle'] = (df_fe['vehicle_price'] < 20000).astype(int)

    df_fe['heavy_vehicle'] = (df_fe['vehicle_weight'] > 30000).astype(int)
    df_fe['light_vehicle'] = (df_fe['vehicle_weight'] < 15000).astype(int)
    df_fe['medium_weight'] = ((df_fe['vehicle_weight'] >= 15000) & (df_fe['vehicle_weight'] <= 30000)).astype(int)

    df_fe['is_large_vehicle'] = (df_fe['vehicle_category'] == 'Large').astype(int)
    df_fe['is_compact_vehicle'] = (df_fe['vehicle_category'] == 'Compact').astype(int)
    df_fe['is_medium_vehicle'] = (df_fe['vehicle_category'] == 'Medium').astype(int)

    # ========================================================================
    # CLAIM CHARACTERISTICS
    # ========================================================================
    df_fe['high_mileage'] = (df_fe['vehicle_mileage'] > 100000).astype(int)
    df_fe['low_mileage'] = (df_fe['vehicle_mileage'] < 50000).astype(int)
    df_fe['very_high_mileage'] = (df_fe['vehicle_mileage'] > 150000).astype(int)
    df_fe['medium_mileage'] = ((df_fe['vehicle_mileage'] >= 50000) & (df_fe['vehicle_mileage'] <= 100000)).astype(int)

    df_fe['frequent_claimer'] = (df_fe['past_num_of_claims'] > 5).astype(int)
    df_fe['moderate_claimer'] = ((df_fe['past_num_of_claims'] >= 1) & (df_fe['past_num_of_claims'] <= 5)).astype(int)
    df_fe['first_time_claimer'] = (df_fe['past_num_of_claims'] == 0).astype(int)
    df_fe['very_frequent_claimer'] = (df_fe['past_num_of_claims'] > 10).astype(int)

    df_fe['large_payout'] = (df_fe['claim_est_payout'] > 5000).astype(int)
    df_fe['medium_payout'] = ((df_fe['claim_est_payout'] >= 2000) & (df_fe['claim_est_payout'] <= 5000)).astype(int)
    df_fe['small_payout'] = (df_fe['claim_est_payout'] < 2000).astype(int)
    df_fe['very_large_payout'] = (df_fe['claim_est_payout'] > 8000).astype(int)

    df_fe['safety_x_prior_claims'] = df_fe['safety_rating'] / (1 + df_fe['past_num_of_claims'])
    df_fe['mileage_x_claims'] = df_fe['vehicle_mileage'] * df_fe['past_num_of_claims']

    # NEW: Claims risk interactions from Doc 8
    df_fe['senior_frequent_claimer'] = df_fe['senior_driver'] * df_fe['frequent_claimer']
    df_fe['low_safety_high_claims'] = ((df_fe['safety_rating'] < 60) & (df_fe['past_num_of_claims'] > 3)).astype(int)

    # ========================================================================
    # RATIO FEATURES
    # ========================================================================
    df_fe['payout_to_price_ratio'] = df_fe['claim_est_payout'] / (df_fe['vehicle_price'] + 1)
    df_fe['severe_damage'] = (df_fe['payout_to_price_ratio'] > 0.3).astype(int)
    df_fe['moderate_damage'] = ((df_fe['payout_to_price_ratio'] >= 0.1) & (df_fe['payout_to_price_ratio'] <= 0.3)).astype(int)
    df_fe['minor_damage'] = (df_fe['payout_to_price_ratio'] < 0.1).astype(int)

    df_fe['income_to_vehicle_price'] = df_fe['annual_income'] / (df_fe['vehicle_price'] + 1)
    df_fe['can_afford_vehicle'] = (df_fe['income_to_vehicle_price'] >= 0.5).astype(int)
    df_fe['expensive_for_income'] = (df_fe['income_to_vehicle_price'] < 0.3).astype(int)

    df_fe['claims_per_year_driving'] = df_fe['past_num_of_claims'] / (df_fe['driving_experience'] + 1)
    df_fe['claim_frequency_high'] = (df_fe['claims_per_year_driving'] > 0.5).astype(int)

    df_fe['safety_to_liability'] = df_fe['safety_rating'] / (df_fe['liab_prct'] + 1)
    df_fe['payout_to_income'] = df_fe['claim_est_payout'] / (df_fe['annual_income'] + 1)
    df_fe['mileage_to_price'] = df_fe['vehicle_mileage'] / (df_fe['vehicle_price'] + 1)
    df_fe['weight_to_price'] = df_fe['vehicle_weight'] / (df_fe['vehicle_price'] + 1)

    # ========================================================================
    # POLICYHOLDER CHARACTERISTICS
    # ========================================================================
    df_fe['high_income'] = (df_fe['annual_income'] > 70000).astype(int)
    df_fe['mid_income'] = ((df_fe['annual_income'] >= 40000) & (df_fe['annual_income'] <= 70000)).astype(int)
    df_fe['low_income'] = (df_fe['annual_income'] < 40000).astype(int)
    df_fe['very_high_income'] = (df_fe['annual_income'] > 100000).astype(int)

    df_fe['high_safety_rating'] = (df_fe['safety_rating'] > 80).astype(int)
    df_fe['low_safety_rating'] = (df_fe['safety_rating'] < 60).astype(int)
    df_fe['very_high_safety'] = (df_fe['safety_rating'] > 90).astype(int)
    df_fe['medium_safety'] = ((df_fe['safety_rating'] >= 60) & (df_fe['safety_rating'] <= 80)).astype(int)

    df_fe['contact_available'] = df_fe['email_or_tel_available']
    df_fe['has_education'] = df_fe['high_education_ind']
    df_fe['recent_move'] = df_fe['address_change_ind']
    df_fe['home_owner'] = (df_fe['living_status'] == 'Own').astype(int)
    df_fe['renter'] = (df_fe['living_status'] == 'Rent').astype(int)
    df_fe['female'] = (df_fe['gender'] == 'F').astype(int)

    # ========================================================================
    # CHANNEL FEATURES
    # ========================================================================
    df_fe['via_broker'] = (df_fe['channel'] == 'Broker').astype(int)
    df_fe['via_online'] = (df_fe['channel'] == 'Online').astype(int)
    df_fe['via_phone'] = (df_fe['channel'] == 'Phone').astype(int)
    df_fe['in_network_repair'] = (df_fe['in_network_bodyshop'] == 'yes').astype(int)
    df_fe['out_network_repair'] = (df_fe['in_network_bodyshop'] == 'no').astype(int)

    # ========================================================================
    # COMPOSITE RECOVERY SCORES
    # ========================================================================
    liability_score = np.sqrt((100 - df_fe['liab_prct']) / 100.0)
    evidence_score_composite = (df_fe['evidence_none'] * 0.0 + df_fe['evidence_weak'] * 0.4 +
                      df_fe['evidence_strong'] * 0.7 + df_fe['evidence_very_strong'] * 1.0)
    clarity_score = df_fe['recovery_case_clarity'] / 3.0
    site_score = df_fe['high_risk_site'] * 0.7 + (1 - df_fe['unknown_site']) * 0.3

    df_fe['recovery_feasibility_score'] = (0.35 * liability_score + 0.30 * df_fe['has_recovery_target'] +
                                           0.20 * evidence_score_composite + 0.10 * clarity_score + 0.05 * site_score)

    # NEW: Alternative recovery potential score from Doc 8
    df_fe['recovery_potential'] = (
        (100 - df_fe['liab_prct']) * 0.4 +
        df_fe['evidence_score'] * 20 * 0.3 +
        df_fe['multicar_binary'] * 30 * 0.2 +
        (df_fe['claim_est_payout'] / 100) * 0.1
    )

    # ========================================================================
    # DOMAIN LOGIC FLAGS (CRITICAL FOR F1)
    # ========================================================================
    df_fe['perfect_case'] = ((df_fe['liab_prct'] < 15) & (df_fe['witness_present'] == 1) &
                             (df_fe['police_report'] == 1) & (df_fe['has_recovery_target'] == 1)).astype(int)

    df_fe['strong_case'] = ((df_fe['liab_prct'] < 25) & (df_fe['evidence_strong'] == 1) &
                            (df_fe['has_recovery_target'] == 1)).astype(int)

    df_fe['good_case'] = ((df_fe['liab_prct'] < 35) & (df_fe['evidence_score'] >= 1) &
                          (df_fe['has_recovery_target'] == 1)).astype(int)

    df_fe['weak_case'] = ((df_fe['liab_prct'] > 40) | (df_fe['is_single_car'] == 1) |
                          (df_fe['evidence_none'] == 1)).astype(int)

    df_fe['no_case'] = ((df_fe['liab_prct'] > 60) | ((df_fe['is_single_car'] == 1) & (df_fe['evidence_none'] == 1))).astype(int)

    df_fe['high_value_opportunity'] = ((df_fe['claim_est_payout'] > 3000) & (df_fe['liab_prct'] < 30) &
                                       (df_fe['has_recovery_target'] == 1)).astype(int)

    df_fe['slam_dunk_case'] = ((df_fe['liab_prct'] < 10) & (df_fe['witness_present'] == 1) &
                               (df_fe['police_report'] == 1) & (df_fe['multicar_binary'] == 1) &
                               (df_fe['high_risk_site'] == 1)).astype(int)

    df_fe['low_liab_high_payout'] = ((df_fe['liab_prct'] < 20) & (df_fe['claim_est_payout'] > 5000)).astype(int)
    df_fe['clear_fault_case'] = ((df_fe['liab_prct'] < 15) & (df_fe['multicar_binary'] == 1)).astype(int)
    df_fe['high_mileage_low_fault'] = ((df_fe['vehicle_mileage'] > 100000) & (df_fe['liab_prct'] < 30)).astype(int)

    # NEW: More interaction flags from Doc 8
    df_fe['low_liab_witness_police'] = ((df_fe['liab_prct'] < 20) & (df_fe['witness_binary'] == 1) &
                                         (df_fe['police_binary'] == 1)).astype(int)
    df_fe['multicar_low_liab'] = ((df_fe['multicar_binary'] == 1) & (df_fe['liab_prct'] < 25)).astype(int)
    df_fe['high_payout_evidence'] = ((df_fe['claim_est_payout'] > 5000) & (df_fe['evidence_score'] >= 1)).astype(int)
    df_fe['severe_damage_low_fault'] = ((df_fe['payout_to_price_ratio'] > 0.3) & (df_fe['liab_prct'] < 30)).astype(int)
    df_fe['minor_damage_high_fault'] = ((df_fe['payout_to_price_ratio'] < 0.1) & (df_fe['liab_prct'] > 50)).astype(int)

    # --- Temporal & Behavior Dynamics ---
    df_fe['claim_early_in_year'] = (df_fe['claim_month'] <= 3).astype(int)
    df_fe['claim_end_of_year'] = (df_fe['claim_month'] >= 10).astype(int)
    df_fe['weekend_parking'] = df_fe['is_weekend'] * (df_fe['accident_site'] == 'Parking Area').astype(int)
    df_fe['winter_claim_high_payout'] = ((df_fe['season'] == 'Winter') & (df_fe['claim_est_payout'] > 5000)).astype(int)

    # --- Vehicle Utilization Proxies (without vehicle_age) ---
    df_fe['mileage_x_weight'] = df_fe['vehicle_mileage'] * df_fe['vehicle_weight']
    df_fe['mileage_per_dollar'] = df_fe['vehicle_mileage'] / (df_fe['vehicle_price'] + 1)
    df_fe['payout_to_weight'] = df_fe['claim_est_payout'] / (df_fe['vehicle_weight'] + 1)

    # --- Policyholder Risk Profile ---
    df_fe['unstable_policyholder'] = ((df_fe['recent_move'] == 1) & (df_fe['renter'] == 1)).astype(int)
    df_fe['financial_stress_risk'] = ((df_fe['expensive_for_income'] == 1) & (df_fe['large_payout'] == 1)).astype(int)
    df_fe['young_driver_highway'] = df_fe['young_driver'] * df_fe['highway_accident']
    df_fe['senior_driver_parking'] = df_fe['senior_driver'] * df_fe['parking_accident']

    # --- Liability & Evidence Interaction Insights ---
    df_fe['low_liab_weak_evidence'] = ((df_fe['liab_prct'] < 20) & (df_fe['evidence_weak'] == 1)).astype(int)
    df_fe['high_liab_strong_evidence'] = ((df_fe['liab_prct'] > 50) & (df_fe['evidence_strong'] == 1)).astype(int)

    # Composite confidence / case quality index
    df_fe['case_confidence_score'] = (
        0.4 * (100 - df_fe['liab_prct']) / 100 +
        0.4 * df_fe['evidence_score'] / 2 +
        0.2 * df_fe['recovery_case_clarity'] / 3
    )

    # --- Statistical Normalization & Percentile Features ---
    for col in ['claim_est_payout', 'vehicle_mileage', 'annual_income']:
        df_fe[f'{col}_z'] = (df_fe[col] - df_fe[col].mean()) / (df_fe[col].std() + 1e-9)

    try:
        df_fe['liab_percentile'] = pd.qcut(df_fe['liab_prct'], 10, labels=False, duplicates='drop')
        df_fe['payout_percentile'] = pd.qcut(df_fe['claim_est_payout'], 10, labels=False, duplicates='drop')
    except Exception:
        df_fe['liab_percentile'] = np.nan
        df_fe['payout_percentile'] = np.nan

    # --- Aggregate / Hybrid Indices ---
    df_fe['case_strength_index'] = df_fe['evidence_score'] * (1 - df_fe['liab_prct'] / 100)
    df_fe['financial_exposure_index'] = (
        (df_fe['claim_est_payout'] / (df_fe['annual_income'] + 1)) * (1 + df_fe['liab_prct'] / 100)
    )
    df_fe['behavioral_risk_index'] = (
        df_fe['claims_per_year_driving'] * (100 - df_fe['safety_rating']) / 100
    )

    return df_fe

print("✓ Feature engineering function defined (190+ features)")

✓ Feature engineering function defined (190+ features)


# pre-modeling with target encoding


In [30]:
print("="*80)
print("Running Feature Engineering on train and test data...")

train_fe = feature_engineer(train_df)
test_fe = feature_engineer(test_df)
print("✓ Feature engineering complete.")

# Define Categorical Feature Lists
features_to_target_encode = [
    'gender', 'living_status', 'accident_site',
    'channel', 'vehicle_category', 'vehicle_color', 'accident_type',
    'in_network_bodyshop', 'season', 'zip_code'
]

# Apply Target Encoding
print(f"\nApplying Smoothed Target Encoding to {len(features_to_target_encode)} features...")
global_mean = train_fe['subrogation'].mean()
categorical_features_for_lgbm = []

for col in features_to_target_encode:
    target_mean = train_fe.groupby(col)['subrogation'].mean()
    category_counts = train_fe.groupby(col).size()
    smoothing = 20

    smoothed_mean = (target_mean * category_counts + global_mean * smoothing) / (category_counts + smoothing)

    new_col_name = f'{col}_target_enc'
    train_fe[new_col_name] = train_fe[col].map(smoothed_mean)
    test_fe[new_col_name] = test_fe[col].map(smoothed_mean)

    test_fe[new_col_name] = test_fe[new_col_name].fillna(global_mean)

    categorical_features_for_lgbm.append(new_col_name)

print("✓ Target encoding complete.")

# Create Final X, y, and X_test
y_all = train_fe['subrogation'].copy()

drop_cols = [
    'subrogation', 'claim_number', 'claim_date', 'year_of_born',
    'witness_present_ind', 'policy_report_filed_ind',
    'vehicle_made_year',  # Bad data quality
    'claim_hour'  # Drop raw hour (we keep rush_hour and late_night flags)
]
drop_cols.extend(features_to_target_encode)

feature_cols = [col for col in train_fe.columns if col not in drop_cols]
X_all = train_fe[feature_cols].copy()
X_test_all = test_fe[feature_cols].copy()

# Apply Label Encoding (if any object columns remain)
other_cat_cols = X_all.select_dtypes(include='object').columns.tolist()
if other_cat_cols:
    print(f"\nApplying Label Encoding to {len(other_cat_cols)} remaining features...")
    for col in other_cat_cols:
        le = LabelEncoder()
        all_values = pd.concat([X_all[col].astype(str), X_test_all[col].astype(str)]).unique()
        le.fit(all_values)
        X_all[col] = le.transform(X_all[col].astype(str))
        X_test_all[col] = le.transform(X_test_all[col].astype(str))
    print("✓ Label encoding complete.")

# Impute NaN values with median
print("\nImputing NaN values with the median from the training data...")
X_all_median = X_all.median()
X_all = X_all.fillna(X_all_median)
X_test_all = X_test_all.fillna(X_all_median)
print("✓ NaN values imputed.")

# Calculate scale_pos_weight
scale_pos_weight = (y_all == 0).sum() / (y_all == 1).sum()

print("\n" + "="*80)
print("PRE-MODELING COMPLETE")
print(f"✓ X_all shape: {X_all.shape}")
print(f"✓ y_all shape: {y_all.shape}")
print(f"✓ X_test_all shape: {X_test_all.shape}")
print(f"✓ Total features: {len(feature_cols)}")
print(f"✓ scale_pos_weight (for F1 score): {scale_pos_weight:.4f}")

Running Feature Engineering on train and test data...
✓ Feature engineering complete.

Applying Smoothed Target Encoding to 10 features...
✓ Target encoding complete.

Applying Label Encoding to 1 remaining features...
✓ Label encoding complete.

Imputing NaN values with the median from the training data...
✓ NaN values imputed.

PRE-MODELING COMPLETE
✓ X_all shape: (17999, 201)
✓ y_all shape: (17999,)
✓ X_test_all shape: (12000, 201)
✓ Total features: 201
✓ scale_pos_weight (for F1 score): 3.3740


# modified optuna with kfold + early stopping




In [ ]:
print("="*80)
print("STEP 1: OPTUNA WITH 5-FOLD CV (PR-AUC + F1 TRACKING)")
print("="*80)

import json
import numpy as np
import optuna
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, average_precision_score, precision_recall_curve

# ---- Toggle which metric to optimize
TARGET = "roc"   # "roc" or "pr"

# ---- Persistent storage
storage = "sqlite:///lgbm_optuna_pr.db"
pruner = optuna.pruners.SuccessiveHalvingPruner(min_resource=1, reduction_factor=3)

study = optuna.create_study(
    direction="maximize",
    study_name="lgbm_kfold_prauc",
    storage=storage,
    load_if_exists=True,
    pruner=pruner,
)

# ---- Objective with 5-fold CV + PR-AUC optimization + F1 tracking
def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 500, 1500),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.15, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 20, 150),
        'max_depth': trial.suggest_int('max_depth', 4, 20),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 2.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 2.0),
        'scale_pos_weight': scale_pos_weight,
        'random_state': RANDOM_STATE,
        'verbose': -1,
        'n_jobs': -1,
    }

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)
    fold_scores = []
    fold_f1s = []  # Track F1 scores

    for fold_idx, (train_idx, val_idx) in enumerate(skf.split(X_all, y_all), start=1):
        X_tr, X_va = X_all.iloc[train_idx], X_all.iloc[val_idx]
        y_tr, y_va = y_all.iloc[train_idx], y_all.iloc[val_idx]

        model = lgb.LGBMClassifier(**params)
        model.fit(
            X_tr, y_tr,
            eval_set=[(X_va, y_va)],
            eval_metric="auc",
            callbacks=[lgb.early_stopping(100, verbose=False)]
        )

        if hasattr(model, "best_iteration_") and model.best_iteration_ is not None:
            proba = model.predict_proba(X_va, num_iteration=model.best_iteration_)[:, 1]
        else:
            proba = model.predict_proba(X_va)[:, 1]

        # Calculate PR-AUC (what we optimize)
        score = average_precision_score(y_va, proba)
        fold_scores.append(score)

        # Also calculate best F1 for this fold (for tracking only)
        prec, rec, thr = precision_recall_curve(y_va, proba)
        f1s = 2 * (prec * rec) / (prec + rec + 1e-12)
        best_f1 = float(np.nanmax(f1s[:-1]))
        fold_f1s.append(best_f1)

        trial.report(float(np.mean(fold_scores)), step=fold_idx)
        if trial.should_prune():
            raise optuna.TrialPruned()

    # Set user attributes to track F1 (won't affect optimization)
    trial.set_user_attr("mean_f1", float(np.mean(fold_f1s)))
    trial.set_user_attr("std_f1", float(np.std(fold_f1s)))

    return float(np.mean(fold_scores))

print("\nRunning Optuna with 5-fold CV (optimizing PR-AUC, tracking F1)...")
print("Progress being written to lgbm_optuna_pr.db\n")

try:
    study.optimize(objective, n_trials=50, show_progress_bar=True)
except KeyboardInterrupt:
    pass

print("\n" + "="*80)
print("OPTUNA RESULTS")
print("="*80)

print(f"✓ Best Mean PR-AUC (5-fold): {study.best_value:.4f}")
print(f"✓ Best Mean F1 (5-fold):     {study.best_trial.user_attrs['mean_f1']:.4f} ± {study.best_trial.user_attrs['std_f1']:.4f}")

print("\n✓ Best parameters:")
for k, v in study.best_params.items():
    print(f"  - {k}: {v}")

# Show top 5 trials with their F1 scores
print("\n📊 Top 5 Trials (by PR-AUC):")
print(f"{'Trial':<8} {'PR-AUC':<10} {'Mean F1':<10} {'Std F1':<10}")
print("="*40)
top_trials = sorted(study.trials, key=lambda t: t.value if t.value is not None else -1, reverse=True)[:5]
for t in top_trials:
    if t.value is not None and 'mean_f1' in t.user_attrs:
        print(f"{t.number:<8} {t.value:<10.4f} {t.user_attrs['mean_f1']:<10.4f} {t.user_attrs['std_f1']:<10.4f}")

# Save best params
best_lgbm_params = study.best_params.copy()
best_lgbm_params.update({
    'scale_pos_weight': scale_pos_weight,
    'random_state': RANDOM_STATE,
    'verbose': -1
})
with open("best_lgbm_params_pr.json", "w") as f:
    json.dump(best_lgbm_params, f)

print("\n✓ Best parameters saved to best_lgbm_params_pr.json")

STEP 1: OPTUNA WITH 5-FOLD CV (PR-AUC + F1 TRACKING)

Running Optuna with 5-fold CV (optimizing PR-AUC, tracking F1)...
Progress being written to lgbm_optuna_pr.db



  0%|          | 0/50 [00:00<?, ?it/s]

# kfold cv

In [21]:
# ============================================================================
# FINAL 5-FOLD CV WITH BEST PARAMETERS — detailed diagnostics & thresholding
# ============================================================================

import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import (
    roc_auc_score, average_precision_score, log_loss, brier_score_loss,
    precision_recall_curve, roc_curve, f1_score, precision_score, recall_score,
    confusion_matrix
)

print("\n" + "="*80)
print("FINAL 5-FOLD CV WITH BEST PARAMETERS (Detailed Analysis)")
print("="*80)

assert 'X_all' in globals() and 'y_all' in globals(), "X_all / y_all not found."
assert 'best_lgbm_params' in globals(), "best_lgbm_params not found."
assert 'RANDOM_STATE' in globals(), "RANDOM_STATE not found."

# Identify categorical features (if any)
if isinstance(X_all, pd.DataFrame):
    cat_features = X_all.select_dtypes(include='category').columns.tolist()
else:
    cat_features = []

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)

# Storage for OOF and per-fold details
oof_probs = np.zeros(len(y_all), dtype=float)
fold_rows = []
pos_rate = float(np.mean(y_all))

print(f"\nClass balance: positives = {pos_rate*100:.2f}%  |  negatives = {(1-pos_rate)*100:.2f}%")

for fold, (tr_idx, va_idx) in enumerate(skf.split(X_all, y_all), 1):
    print(f"\n{'='*40}\nFold {fold}/5\n{'='*40}")

    X_tr, X_va = X_all.iloc[tr_idx], X_all.iloc[va_idx]
    y_tr, y_va = y_all.iloc[tr_idx], y_all.iloc[va_idx]

    model = lgb.LGBMClassifier(**best_lgbm_params)

    # Train with early stopping via callbacks (compatible with older LightGBM)
    model.fit(
        X_tr, y_tr,
        eval_set=[(X_va, y_va)],
        eval_metric="auc",
        callbacks=[lgb.early_stopping(200, verbose=False), lgb.log_evaluation(period=0)],
        categorical_feature=cat_features if len(cat_features) > 0 else "auto",
    )

    # Predict using best_iteration_ if available
    if hasattr(model, "best_iteration_") and model.best_iteration_ is not None:
        probs = model.predict_proba(X_va, num_iteration=model.best_iteration_)[:, 1]
        best_iters = int(model.best_iteration_)
    else:
        probs = model.predict_proba(X_va)[:, 1]
        best_iters = int(best_lgbm_params.get('n_estimators', 0)) or None

    # Store OOF probabilities
    oof_probs[va_idx] = probs

    # ---- Fold metrics (threshold-free)
    fold_roc = roc_auc_score(y_va, probs)
    fold_ap  = average_precision_score(y_va, probs)
    # log_loss needs probs in (0,1); clip to be safe
    fold_logloss = log_loss(y_va, np.clip(probs, 1e-7, 1-1e-7))
    fold_brier   = brier_score_loss(y_va, probs)
    fpr, tpr, _  = roc_curve(y_va, probs)
    fold_ks      = float(np.max(tpr - fpr))

    # ---- Fold-optimal threshold by PR curve (maximize F1)
    prec, rec, thr = precision_recall_curve(y_va, probs)
    f1s = 2 * (prec * rec) / (prec + rec + 1e-12)
    # thresholds has length len(prec)-1; align indices
    best_idx = int(np.nanargmax(f1s[:-1])) if len(f1s) > 1 else 0
    fold_thresh = float(thr[max(0, best_idx)])

    y_pred_fold = (probs >= fold_thresh).astype(int)
    fold_f1     = f1_score(y_va, y_pred_fold)
    fold_prec   = precision_score(y_va, y_pred_fold, zero_division=0)
    fold_rec    = recall_score(y_va, y_pred_fold, zero_division=0)

    print(f"  AUC-ROC: {fold_roc:.4f} | PR-AUC (AP): {fold_ap:.4f} | LogLoss: {fold_logloss:.4f} | Brier: {fold_brier:.4f} | KS: {fold_ks:.4f}")
    print(f"  Fold-opt Threshold: {fold_thresh:.4f} | F1: {fold_f1:.4f} | Precision: {fold_prec:.4f} | Recall: {fold_rec:.4f} | Best iters: {best_iters}")

    fold_rows.append({
        "Fold": fold,
        "ROC_AUC": fold_roc,
        "PR_AUC": fold_ap,
        "LogLoss": fold_logloss,
        "Brier": fold_brier,
        "KS": fold_ks,
        "Fold_Threshold": fold_thresh,
        "Fold_F1": fold_f1,
        "Fold_Precision": fold_prec,
        "Fold_Recall": fold_rec,
        "Best_Iterations": best_iters,
    })

# =========================
# OOF Summary (threshold-free)
# =========================
print(f"\n{'='*80}")
print("OOF SUMMARY (Threshold-Free)")
print(f"{'='*80}")

oof_roc = roc_auc_score(y_all, oof_probs)
oof_ap  = average_precision_score(y_all, oof_probs)
oof_logloss = log_loss(y_all, np.clip(oof_probs, 1e-7, 1-1e-7))
oof_brier   = brier_score_loss(y_all, oof_probs)
fpr, tpr, _ = roc_curve(y_all, oof_probs)
oof_ks      = float(np.max(tpr - fpr))

print(f"✓ OOF AUC-ROC: {oof_roc:.4f}")
print(f"✓ OOF PR-AUC (AP): {oof_ap:.4f}")
print(f"✓ OOF LogLoss: {oof_logloss:.4f}")
print(f"✓ OOF Brier score: {oof_brier:.4f}")
print(f"✓ OOF KS statistic: {oof_ks:.4f}")

# =========================
# Global OOF threshold search (maximize F1)
# =========================
print(f"\n{'='*80}")
print("GLOBAL OOF THRESHOLD OPTIMIZATION (by F1)")
print(f"{'='*80}")

prec, rec, thr = precision_recall_curve(y_all, oof_probs)
f1s = 2 * (prec * rec) / (prec + rec + 1e-12)
best_idx = int(np.nanargmax(f1s[:-1])) if len(f1s) > 1 else 0
global_thresh = float(thr[max(0, best_idx)])
global_f1 = float(f1s[best_idx])

global_preds = (oof_probs >= global_thresh).astype(int)
global_prec = precision_score(y_all, global_preds, zero_division=0)
global_rec  = recall_score(y_all, global_preds, zero_division=0)
tn, fp, fn, tp = confusion_matrix(y_all, global_preds).ravel()
specificity = tn / (tn + fp) if (tn + fp) > 0 else 0.0

print(f"✅ Best Global Threshold: {global_thresh:.4f}")
print(f"✅ OOF F1 at Global Threshold: {global_f1:.4f}")
print(f"   Precision: {global_prec:.4f} | Recall: {global_rec:.4f} | Specificity: {specificity:.4f}")
print(f"   Confusion Matrix @ threshold {global_thresh:.4f} -> TN:{tn}  FP:{fp}  FN:{fn}  TP:{tp}")

# =========================
# Per-fold table & dispersion
# =========================
fold_df = pd.DataFrame(fold_rows)
pd.set_option('display.float_format', lambda x: f"{x:.4f}")

print("\n📊 Per-Fold Results:")
print(fold_df.to_string(index=False))

def _mean_std(col):
    return f"{fold_df[col].mean():.4f} ± {fold_df[col].std():.4f}"

print("\nSummary (mean ± std across folds):")
print(f"  ROC_AUC       : {_mean_std('ROC_AUC')}")
print(f"  PR_AUC (AP)   : {_mean_std('PR_AUC')}")
print(f"  LogLoss       : {_mean_std('LogLoss')}")
print(f"  Brier         : {_mean_std('Brier')}")
print(f"  KS            : {_mean_std('KS')}")
print(f"  Fold_F1       : {_mean_std('Fold_F1')}")
print(f"  Fold_Threshold: {_mean_std('Fold_Threshold')}")
print(f"  Best_Iterations: {_mean_std('Best_Iterations') if fold_df['Best_Iterations'].notna().any() else 'n/a'}")

# Final recommended threshold for deployment (from OOF global search)
final_threshold = global_thresh
print(f"\n🎯 Recommended Operating Threshold (from OOF): {final_threshold:.4f}")



FINAL 5-FOLD CV WITH BEST PARAMETERS (Detailed Analysis)

Class balance: positives = 22.86%  |  negatives = 77.14%

Fold 1/5
  AUC-ROC: 0.8357 | PR-AUC (AP): 0.5884 | LogLoss: 0.4527 | Brier: 0.1509 | KS: 0.5172
  Fold-opt Threshold: 0.5381 | F1: 0.5882 | Precision: 0.5237 | Recall: 0.6707 | Best iters: 72

Fold 2/5
  AUC-ROC: 0.8378 | PR-AUC (AP): 0.6114 | LogLoss: 0.4606 | Brier: 0.1539 | KS: 0.5258
  Fold-opt Threshold: 0.6001 | F1: 0.5943 | Precision: 0.5508 | Recall: 0.6452 | Best iters: 62

Fold 3/5
  AUC-ROC: 0.8530 | PR-AUC (AP): 0.6143 | LogLoss: 0.4496 | Brier: 0.1495 | KS: 0.5479
  Fold-opt Threshold: 0.5643 | F1: 0.6200 | Precision: 0.5440 | Recall: 0.7205 | Best iters: 69

Fold 4/5
  AUC-ROC: 0.8321 | PR-AUC (AP): 0.5946 | LogLoss: 0.4653 | Brier: 0.1556 | KS: 0.5157
  Fold-opt Threshold: 0.5859 | F1: 0.5875 | Precision: 0.5343 | Recall: 0.6525 | Best iters: 68

Fold 5/5
  AUC-ROC: 0.8372 | PR-AUC (AP): 0.5917 | LogLoss: 0.4603 | Brier: 0.1539 | KS: 0.5204
  Fold-opt Thre

# threshold selection

In [24]:
print("\n" + "="*80)
print("STEP 3: FIND OPTIMAL THRESHOLD ON CALIBRATED VALIDATION")
print("="*80)

# Split for threshold finding: 85% train/cal, 15% validation
X_train_cal, X_val_thresh, y_train_cal, y_val_thresh = train_test_split(
    X_all, y_all,
    test_size=0.15,
    random_state=RANDOM_STATE,
    stratify=y_all
)

# Further split train_cal: 85% train, 15% calibration
X_train_sub, X_cal_sub, y_train_sub, y_cal_sub = train_test_split(
    X_train_cal, y_train_cal,
    test_size=0.15 / 0.85,  # To get final 72.25% train, 12.75% cal, 15% val
    random_state=RANDOM_STATE,
    stratify=y_train_cal
)

print(f"Train: {X_train_sub.shape[0]} samples")
print(f"Calibration: {X_cal_sub.shape[0]} samples")
print(f"Validation (for threshold): {X_val_thresh.shape[0]} samples")

# Internal ES split
X_tr_es, X_es, y_tr_es, y_es = train_test_split(
    X_train_sub, y_train_sub,
    test_size=0.10,
    random_state=RANDOM_STATE,
    stratify=y_train_sub
)

cat_features = X_all.select_dtypes(include='category').columns.tolist() if isinstance(X_all, pd.DataFrame) else []

# Train base model
print("\nTraining base model with early stopping...")
base_model = lgb.LGBMClassifier(**best_lgbm_params)
base_model.fit(
    X_tr_es, y_tr_es,
    eval_set=[(X_es, y_es)],
    eval_metric="auc",
    callbacks=[lgb.early_stopping(200, verbose=False), lgb.log_evaluation(period=0)],
    categorical_feature=cat_features if len(cat_features) > 0 else "auto"
)
print(f"✓ Base model trained (best_iteration: {base_model.best_iteration_})")

# Calibrate
print("Calibrating model...")
calibrated_model = CalibratedClassifierCV(
    base_model,
    method='sigmoid',
    cv='prefit'
)
calibrated_model.fit(X_cal_sub, y_cal_sub)
print("✓ Model calibrated")

# Get calibrated probabilities on validation set
val_probs_calibrated = calibrated_model.predict_proba(X_val_thresh)[:, 1]

print(f"\nValidation probability statistics:")
print(f"  Mean: {val_probs_calibrated.mean():.4f}")
print(f"  Std:  {val_probs_calibrated.std():.4f}")
print(f"  Median: {np.median(val_probs_calibrated):.4f}")

# Find optimal threshold using multiple methods
print("\n" + "="*80)
print("THRESHOLD SEARCH ON CALIBRATED VALIDATION SET")
print("="*80)

from sklearn.metrics import precision_recall_curve, f1_score

# Method 1: PR Curve
prec, rec, thr = precision_recall_curve(y_val_thresh, val_probs_calibrated)
f1s = 2 * (prec * rec) / (prec + rec + 1e-12)
best_idx = np.argmax(f1s[:-1])
threshold_pr = float(thr[best_idx])
f1_pr = float(f1s[best_idx])

# Method 2: Grid Search
thresholds_grid = np.arange(0.15, 0.50, 0.01)
f1_scores_grid = []
for t in thresholds_grid:
    preds = (val_probs_calibrated >= t).astype(int)
    f1_scores_grid.append(f1_score(y_val_thresh, preds))

best_idx_grid = np.argmax(f1_scores_grid)
threshold_grid = float(thresholds_grid[best_idx_grid])
f1_grid = float(f1_scores_grid[best_idx_grid])

# Method 3: Match training distribution
positive_rate = (y_all == 1).sum() / len(y_all)
threshold_percentile = float(np.percentile(val_probs_calibrated, (1 - positive_rate) * 100))
preds_pct = (val_probs_calibrated >= threshold_percentile).astype(int)
f1_pct = float(f1_score(y_val_thresh, preds_pct))

print(f"\n{'Method':<25} {'Threshold':<12} {'Val F1':<10} {'Precision':<12} {'Recall':<10}")
print("="*80)

results = [
    ("PR Curve", threshold_pr, f1_pr),
    ("Grid Search", threshold_grid, f1_grid),
    ("Percentile Match", threshold_percentile, f1_pct),
]

best_f1 = 0
for method_name, thresh, f1 in results:
    preds = (val_probs_calibrated >= thresh).astype(int)
    prec = precision_score(y_val_thresh, preds, zero_division=0)
    rec = recall_score(y_val_thresh, preds, zero_division=0)

    marker = ""
    if f1 > best_f1:
        best_f1 = f1
        optimal_threshold = thresh
        best_method = method_name
        marker = " ← BEST"

    print(f"{method_name:<25} {thresh:<12.4f} {f1:<10.4f} {prec:<12.4f} {rec:<10.4f}{marker}")

print("="*80)

print(f"\n✅ SELECTED: {best_method}")
print(f"✅ Threshold: {optimal_threshold:.4f}")
print(f"✅ Validation F1: {best_f1:.4f}")

preds_final = (val_probs_calibrated >= optimal_threshold).astype(int)
final_prec = precision_score(y_val_thresh, preds_final, zero_division=0)
final_rec = recall_score(y_val_thresh, preds_final, zero_division=0)

print(f"\nPerformance at selected threshold:")
print(f"  Precision: {final_prec:.4f}")
print(f"  Recall:    {final_rec:.4f}")
print(f"  F1:        {best_f1:.4f}")

print(f"\n{'='*80}")
print("✓ Threshold selection complete")
print(f"{'='*80}\n")


STEP 3: FIND OPTIMAL THRESHOLD ON CALIBRATED VALIDATION
Train: 12599 samples
Calibration: 2700 samples
Validation (for threshold): 2700 samples

Training base model with early stopping...
✓ Base model trained (best_iteration: 60)
Calibrating model...
✓ Model calibrated

Validation probability statistics:
  Mean: 0.2198
  Std:  0.2130
  Median: 0.1099

THRESHOLD SEARCH ON CALIBRATED VALIDATION SET

Method                    Threshold    Val F1     Precision    Recall    
PR Curve                  0.3093       0.5954     0.5311       0.6775     ← BEST
Grid Search               0.3100       0.5953     0.5319       0.6759    
Percentile Match          0.4055       0.5765     0.5761       0.5770    

✅ SELECTED: PR Curve
✅ Threshold: 0.3093
✅ Validation F1: 0.5954

Performance at selected threshold:
  Precision: 0.5311
  Recall:    0.6775
  F1:        0.5954

✓ Threshold selection complete



# final model training


In [25]:
print("\n" + "="*80)
print("STEP 4: RETRAIN FINAL MODEL ON 85% DATA")
print("="*80)

# Split: 85% train, 15% calibration
X_train_final, X_cal_final, y_train_final, y_cal_final = train_test_split(
    X_all, y_all,
    test_size=0.15,
    random_state=RANDOM_STATE,
    stratify=y_all
)

print(f"Final train: {X_train_final.shape[0]} samples")
print(f"Final calibration: {X_cal_final.shape[0]} samples")

# Internal ES split from train_final (10% for early stopping)
X_tr_final, X_es_final, y_tr_final, y_es_final = train_test_split(
    X_train_final, y_train_final,
    test_size=0.10,
    random_state=RANDOM_STATE,
    stratify=y_train_final
)

cat_features = X_all.select_dtypes(include='category').columns.tolist() if isinstance(X_all, pd.DataFrame) else []

# Train with early stopping
print("\nTraining final model with early stopping...")
final_base_model = lgb.LGBMClassifier(**best_lgbm_params)
final_base_model.fit(
    X_tr_final, y_tr_final,
    eval_set=[(X_es_final, y_es_final)],
    eval_metric="auc",
    callbacks=[lgb.early_stopping(200, verbose=False), lgb.log_evaluation(period=0)],
    categorical_feature=cat_features if len(cat_features) > 0 else "auto"
)
print(f"✓ Final model trained (best_iteration: {final_base_model.best_iteration_})")

# Calibrate
print("Calibrating final model...")
final_calibrated_model = CalibratedClassifierCV(
    final_base_model,
    method='sigmoid',
    cv='prefit'
)
final_calibrated_model.fit(X_cal_final, y_cal_final)
print("✓ Final model calibrated")

# Use threshold from Step 3 (calibrated validation)
print(f"\n✓ Using threshold from Step 3: {optimal_threshold:.4f}")
print(f"   Validation F1: {best_f1:.4f}")
print("="*80 + "\n")


STEP 4: RETRAIN FINAL MODEL ON 85% DATA
Final train: 15299 samples
Final calibration: 2700 samples

Training final model with early stopping...
✓ Final model trained (best_iteration: 46)
Calibrating final model...
✓ Final model calibrated

✓ Using threshold from Step 3: 0.3093
   Validation F1: 0.5954



# prediction

In [26]:

print("="*80)
print("STEP 5: GENERATING PREDICTIONS ON TEST SET")
print("="*80)

print(f"\nGenerating calibrated predictions for {X_test_all.shape[0]} test samples...")

# Get calibrated probabilities
test_probabilities = final_calibrated_model.predict_proba(X_test_all)[:, 1]

# Apply threshold from Step 3
test_predictions = (test_probabilities >= optimal_threshold).astype(int)

print(f"✓ Predictions generated")
print(f"✓ Using threshold: {optimal_threshold:.4f}")
print(f"\nPrediction distribution:")
print(f"  - Class 0 (No subrogation): {(test_predictions == 0).sum()} ({(test_predictions == 0).sum() / len(test_predictions) * 100:.1f}%)")
print(f"  - Class 1 (Subrogation):    {(test_predictions == 1).sum()} ({(test_predictions == 1).sum() / len(test_predictions) * 100:.1f}%)")

print(f"\nProbability statistics:")
print(f"  - Mean: {test_probabilities.mean():.4f}")
print(f"  - Std:  {test_probabilities.std():.4f}")
print(f"  - Min:  {test_probabilities.min():.4f}")
print(f"  - Max:  {test_probabilities.max():.4f}")

# Create submission
submission = pd.DataFrame({
    'claim_number': test_fe['claim_number'],
    'subrogation': test_predictions
})

output_filename = 'TriGuard_pr_calval_thresh_3093.csv'
submission.to_csv(output_filename, index=False)

print(f"\n{'='*90}")
print(f"✓ SUBMISSION FILE SAVED: {output_filename}")
print(f"{'='*90}")

# Download
try:
    from google.colab import files
    files.download(output_filename)
    print("✓ File downloaded!")
except:
    print("(File saved locally)")

print(f"\n{'='*90}")
print("PIPELINE COMPLETE!")
print(f"Expected F1: 0.595-0.605 (based on calibrated validation)")
print(f"{'='*90}\n")

STEP 5: GENERATING PREDICTIONS ON TEST SET

Generating calibrated predictions for 12000 test samples...
✓ Predictions generated
✓ Using threshold: 0.3093

Prediction distribution:
  - Class 0 (No subrogation): 8200 (68.3%)
  - Class 1 (Subrogation):    3800 (31.7%)

Probability statistics:
  - Mean: 0.2323
  - Std:  0.2166
  - Min:  0.0322
  - Max:  0.7197

✓ SUBMISSION FILE SAVED: TriGuard_pr_calval_thresh_3093.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✓ File downloaded!

PIPELINE COMPLETE!
Expected F1: 0.595-0.605 (based on calibrated validation)



# choosing threshold


In [10]:
print("="*80)
print("THRESHOLD ANALYSIS - PREVIEW DISTRIBUTIONS")
print("="*80)

import pandas as pd
import numpy as np

# Your test probabilities (already calculated)
print(f"\nTest probability statistics:")
print(f"  Mean: {test_probabilities.mean():.4f}")
print(f"  Median: {np.median(test_probabilities):.4f}")
print(f"  Std: {test_probabilities.std():.4f}")
print(f"  25th percentile: {np.percentile(test_probabilities, 25):.4f}")
print(f"  75th percentile: {np.percentile(test_probabilities, 75):.4f}")

# Test multiple thresholds and show distributions
thresholds_to_test = [0.25, 0.30, 0.35, 0.40, 0.45, 0.50, 0.5719]

print(f"\n{'='*80}")
print(f"{'Threshold':<12} {'N Positive':<12} {'% Positive':<12} {'N Negative':<12} {'% Negative':<12}")
print(f"{'='*80}")

results = []
for thresh in thresholds_to_test:
    preds = (test_probabilities >= thresh).astype(int)
    n_pos = preds.sum()
    n_neg = len(preds) - n_pos
    pct_pos = n_pos / len(preds) * 100
    pct_neg = 100 - pct_pos

    results.append({
        'threshold': thresh,
        'n_positive': n_pos,
        'pct_positive': pct_pos,
        'n_negative': n_neg,
        'pct_negative': pct_neg
    })

    marker = ""
    if thresh == 0.3540:
        marker = " ← Validation optimal"
    elif thresh == 0.5719:
        marker = " ← OOF threshold (current)"

    print(f"{thresh:<12.4f} {n_pos:<12} {pct_pos:<12.1f} {n_neg:<12} {pct_neg:<12.1f}{marker}")

print(f"{'='*80}\n")

# Show which look most reasonable
print("📊 Analysis:")
print(f"  - Training data has {(y_all == 1).sum() / len(y_all) * 100:.1f}% positive class")
print(f"  - Validation @0.3540: F1 = 0.5912 (53.3% precision, 66.5% recall)")
print(f"  - Validation @0.5719: F1 = 0.3936 (66.0% precision, 28.0% recall)")
print(f"\n  💡 Thresholds around 0.30-0.40 look most reasonable")
print(f"     (25-35% positive predictions, similar to training ratio)")

# Ask which ones to save
print(f"\n{'='*80}")
print("Which thresholds do you want to save as CSV files?")
print("Options: 0.25, 0.30, 0.35, 0.40, 0.45, 0.50, 0.5719")
print("(You can pick multiple, separated by commas)")
print(f"{'='*80}")

THRESHOLD ANALYSIS - PREVIEW DISTRIBUTIONS

Test probability statistics:
  Mean: 0.2325
  Median: 0.1305
  Std: 0.2148
  25th percentile: 0.0493
  75th percentile: 0.3976

Threshold    N Positive   % Positive   N Negative   % Negative  
0.2500       4383         36.5         7617         63.5        
0.3000       3888         32.4         8112         67.6        
0.3500       3432         28.6         8568         71.4        
0.4000       2976         24.8         9024         75.2        
0.4500       2587         21.6         9413         78.4        
0.5000       2148         17.9         9852         82.1        
0.5719       1474         12.3         10526        87.7         ← OOF threshold (current)

📊 Analysis:
  - Training data has 22.9% positive class
  - Validation @0.3540: F1 = 0.5912 (53.3% precision, 66.5% recall)
  - Validation @0.5719: F1 = 0.3936 (66.0% precision, 28.0% recall)

  💡 Thresholds around 0.30-0.40 look most reasonable
     (25-35% positive predictions, s